# Projeto 2 - Ciência dos Dados

	



A equipe de marketing do presidente do Estados Unidos da América, Donald Trump, te contratou para um trabalho importante relativa à sua campanha presidencial para as eleições de 2020.<br /><br />
Para este trabalho, sua equipe decidiu contratar dois cientistas de dados para analisar os tweets no twitter referentes ao Donald Trump e classificá-los entre relevante para a sua campanha presidencial (tweets no qual os usuários criticam ou apoiam o presidente) ou irrelevantes (tweets onde o foco não é o Donald Trump ou, por exemplo, chamadas de notícias, lugar onde opiniões não estão presentes) com o intuito de ajudar a equipe de marketing a direcionar sua campanha visto que, com as críticas, poderão ser encontrados pontos a melhorar e, com os elogios, pontos que continuarão a investir para agradar o povo, assim, maximizando a eficiência de sua campanha.<br /><br />
Os cientistas de dados, ao verem essa proposta, decidiram utilizar seus conhecimentos da ferramenta de classificação Naive-Bayes para ensinarem seu programa a classificar os tweets como relevante ou não relevante ao assunto em questão. Para isso, terão de implementar uma versão do classificador que aprende a relevância dos tweets a partir de uma base de treinamento e irão comparar a classificação do programa com a manual, checando a performance do classificador. Feito isso, o classificador já estará pronto para classificar quaisquer tweets em relação ao assunto automaticamente.


## Naive-Bayes 

Explicar como funciona naive bayes e dar exemplos de uso (critério de nota)


Nome: Arthur Carvalho

Nome: Guilherme Lunetta

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [3]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emoji
!pip install emoji_list

In [4]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import emoji
import emoji_list

___
## Autenticando no  Twitter

* Conta: @tuca1209

In [5]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: Arthur FC

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [6]:
#Produto escolhido:
produto = 'Trump'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [7]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []


for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
            break

            
#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel

In [8]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

Parte do excel como prova

<img src='Classifica_mao.png' width='1000'>

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

#### Funções utilizadas no projeto

In [9]:
import re

dados = pd.read_excel('Trump.xlsx')
tweets = dados.iloc[0:,0]

def limpar_caracteres(tabela, titulo):
    a = tabela[titulo]
    a = a.str.lower()

    itens = ['.', ':', ';', '"', "'", '?', '(', ')', '[',']',',', '\n', '\t']
    i2 = ['?', '!']
    emojis = emoji_list.all_emoji
    for emoji in emojis:
        try:
            a = a.str.replace(emoji," {} ".format(emoji) )
        except:
            pass
    for e in itens:
        a = a.str.replace(e,'')
    for e in i2:
        a = a.str.replace(e,' {} '.format(e))

    a = a.str.replace('é','e')
    a = a.str.replace('ê','e')
    a = a.str.replace('á','a')
    a = a.str.replace('ã','a')
    a = a.str.replace('ô','o')
    a = a.str.replace('ó','o')
    a = a.str.replace('ú','u')
    a = a.str.replace('ç','c')
    a = a.str.replace('í','i')
    a = a.str.replace('@',' @')
    a = a.str.replace(',','.')
        
    tabela[titulo] = a

    return tabela

titulo = "Treinamento"
tabela = limpar_caracteres(dados, titulo)

def split_tweet(tabela, titulo):
    a = tabela[titulo].str.split(' ')

    nova = pd.DataFrame()
    nova['palavras'] = []

    for lista in a:
        for e in ['#','' , ' ', 'rt']:
            while lista.count(e) != 0:
                lista.remove(e)

        for palavra in lista:
            if '@' in palavra and palavra != '@realDonaldTrump':
                lista.remove(palavra) 
            if palavra[:4] == 'http':
                lista.remove(palavra)
        nova = nova.append({'palavras': lista}, ignore_index=True)
                
    return nova

nova = split_tweet(dados, titulo)


def tabela_palavras(tabela):
    a = tabela['palavras']
    prob = pd.DataFrame()
    prob['palavras'] = []
    for lista in a:
        for e in lista:
            prob = prob.append({'palavras': e}, ignore_index=True)
    
    return prob['palavras']

def tabela_tudo(tabela1, tabela2):
    a = tabela1['palavras']
    b = tabela2['palavras']
    tudo = pd.DataFrame()
    tudo['palavras'] = []
    for lista in a:
        for e in lista:
            tudo = tudo.append({'palavras': e}, ignore_index=True)
    for lista in b:
        for e in lista:
            tudo = tudo.append({'palavras': e}, ignore_index=True)
    
    return tudo

Separando os tweets entre relevantes e irrelevantes, em DataFrames separados

In [10]:
relevante = dados.loc[(dados['Relevancia'])==1]
irrelevante = dados.loc[(dados['Relevancia'])==0]

Lendo o excel de Teste

In [11]:
trump_teste = pd.read_excel('TrumpTeste.xlsx')

Preparando o terreno para a suavização de Laplace

In [12]:
palavras_rel = tabela_palavras(split_tweet(limpar_caracteres(relevante, 'Treinamento'), 'Treinamento'))
num_rel = palavras_rel.value_counts()
len_rel = len(num_rel)

palavras_irrel = tabela_palavras(split_tweet(limpar_caracteres(irrelevante, 'Treinamento'), 'Treinamento'))
num_irrel = palavras_irrel.value_counts()
len_irrel = len(num_irrel)

teste = split_tweet(limpar_caracteres(trump_teste, 'Teste'), 'Teste')
teste = teste.join(trump_teste['Relevancia'])

tudo = tabela_tudo(split_tweet(limpar_caracteres(relevante, 'Treinamento'), 'Treinamento'),split_tweet(limpar_caracteres(irrelevante, 'Treinamento'), 'Treinamento'))
#Total de palavras diferentes que aparecem na base de treinamento para o LaPlace.
total_palavras = len(tudo['palavras'].value_counts())

C:\Users\Usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [13]:
i = 0
teste["Avaliacao"] = pd.Series()

for linha in teste["palavras"]:
    p_um = 1
    p_zero = 1
    p_naive = ''
    for palavra in linha:
        if palavra in num_rel:
            #laplace
            p_um *= (num_rel[palavra] + 1)/(len_rel + total_palavras)
            
            #print(p_um - p_zero)
        else:
            p_um *= 1/(len_rel + total_palavras)
            
            
        if palavra in num_irrel:
            #laplace
            
            p_zero *= (num_irrel[palavra] + 1)/(len_irrel + total_palavras)
            #print(p_um - p_zero)
        else:
            
            p_zero *= 1/(len_irrel + total_palavras)
            
    if p_um > p_zero:
        p_naive = '1'
        teste["Avaliacao"][i] = p_naive
        

    elif p_um < p_zero:
        p_naive = '0'
        teste["Avaliacao"][i] = p_naive
        
    i+=1

C:\Users\Usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Checando se o código funcionou

In [15]:
teste.head(5)

,palavras,Relevancia,Avaliacao
0,"[se, preocupe, com, trump, macron, !, sua, apr...",0,0.0
1,"[quem, ainda, tem, alguma, esperanca, da, gran...",0,0.0
2,"[o, filme, e, sobre, o, trump]",0,0.0
3,"[vai, deixar, o, patrao, trump, bravinho, hein]",1,1.0
4,"[ninguem, liga, pra, vcs, mais, nao, perdendo,...",0,0.0


Analisando a eficiência do classificador a partir de uma crosstab

In [16]:
valid = pd.crosstab(teste["Relevancia"], teste['Avaliacao'])
valid

Avaliacao,0.0,1.0
Relevancia,,
0,121,3
1,62,14


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**